In [1]:
#####------make sure you set the project_name and model_version accurately------#####
project_name = 'lentil'
model_version = 'v1'

In [2]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json
import os
import sys

In [4]:
config = json.load(open('/content/drive/MyDrive/projects/{0}/models/{1}/config.json'.format(project_name, model_version),))
config

{'anomalous_factor': 15,
 'classes': ['Stalk', 'Broken', 'Pod', 'Seeds', 'Normal'],
 'colors': ['Teal', 'Cyan', 'Magenta', 'OrangeRed', 'Chartreuse'],
 'data_version': 'v1',
 'description': 'The model is trained with data version v1',
 'dispaly_format': 'letter',
 'eval_steps': 500,
 'letters': ['T', 'B', 'P', 'S', 'N'],
 'model_version': 'v1',
 'patch_overlap': 40,
 'patch_size': 640,
 'project_name': 'lentil',
 'remove_anomalies': 'True',
 'test_images': ['gomicro'],
 'test_split': 0.2,
 'threshold': 0.5,
 'train_steps': 10000}

In [5]:
base_path = '/content/drive/MyDrive/projects'

In [6]:
base_model_path = os.path.join(base_path, config['project_name'], 'models', config['model_version'])
base_model_path

'/content/drive/MyDrive/projects/lentil/models/v1'

**Load Model**

In [7]:
import tensorflow as tf

In [8]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(os.path.join(base_model_path, 'model', 'saved_model'))

**Test**

In [9]:
patch_size = int(config['patch_size'])
patch_overlap = int(config['patch_overlap'])
dispaly_format = config['dispaly_format']
remove_anomalies = config['remove_anomalies'] == 'True'
anomalous_factor = int(config['anomalous_factor'])
colors = config['colors']
letters = config['letters']
classes = config['classes']
threshold = config['threshold']
patch_size, patch_overlap, dispaly_format, remove_anomalies, anomalous_factor, colors, letters, classes, threshold

(640,
 40,
 'letter',
 True,
 15,
 ['Teal', 'Cyan', 'Magenta', 'OrangeRed', 'Chartreuse'],
 ['T', 'B', 'P', 'S', 'N'],
 ['Stalk', 'Broken', 'Pod', 'Seeds', 'Normal'],
 0.5)

In [10]:
font_path = '/content/drive/MyDrive/code/fonts/Roboto/Roboto-Bold.ttf'

In [11]:
sys.path.append(os.path.abspath('/content/drive/MyDrive/code/detection/shared'))

In [12]:
from detection_utility import get_detections, remove_anomalous_boxes
from display_utility import group_by_label, draw_detections
from PIL import Image
from datetime import datetime

In [13]:
base_results_path = os.path.join(base_model_path, 'results', datetime.now().strftime("%Y%m%d_%H%M"))
base_results_path

'/content/drive/MyDrive/projects/lentil/models/v1/results/20211124_0806'

In [14]:
for test_dir in config['test_images']:
  test_dir_path = os.path.join(base_path, config['project_name'], 'data', 'test_images', test_dir)
  test_imgs = os.listdir(test_dir_path)
  results_path = os.path.join(base_results_path, test_dir)
  os.makedirs(results_path, exist_ok=True)
  for test_img in test_imgs:
    img = Image.open(os.path.join(test_dir_path, test_img))
    detections = get_detections(model, img, patch_size, patch_overlap, threshold=threshold, remove_overlap = False, overlap_threshold=0.8)
    if remove_anomalies:
      detections = remove_anomalous_boxes(detections, anomalous_factor)
    stats = group_by_label(detections)
    draw_detections(img, detections, colors, letters, font_path, 32, dispaly_format)
    img.save(os.path.join(results_path, test_img))
    
    file_name = os.path.splitext(test_img)[0]

    stats_path = os.path.join(results_path, file_name + '_stats.json')
    with open(stats_path, 'w') as f:
      json.dump(stats, f)

    detections_path = os.path.join(results_path, file_name + '_detections.json')
    with open(detections_path, 'w') as f:
      json.dump(str(detections), f)      

h_s:  0  h_overlap:  40  w_s:  0  w_overlap:  40
current patch detections:  1
h_s:  0  h_overlap:  40  w_s:  600  w_overlap:  40
current patch detections:  42
previous patch detections:  1
previous patch detections after w-merge:  1
current patch detections after w-merge:  42
h_s:  0  h_overlap:  40  w_s:  1200  w_overlap:  40
current patch detections:  110
previous patch detections:  42
previous patch detections after w-merge:  37
current patch detections after w-merge:  104
h_s:  0  h_overlap:  40  w_s:  1800  w_overlap:  40
current patch detections:  105
previous patch detections:  104
previous patch detections after w-merge:  100
current patch detections after w-merge:  101
h_s:  0  h_overlap:  40  w_s:  2400  w_overlap:  40
current patch detections:  94
previous patch detections:  101
previous patch detections after w-merge:  96
current patch detections after w-merge:  89
h_s:  0  h_overlap:  40  w_s:  3000  w_overlap:  40
current patch detections:  8
previous patch detections:  8